# Projeto: Esgotamento Sanitário na Região Nordeste (2010)

### Autores:

- Gabriel Igor Queiroz Costa - gabriel.igorq@gmail.com
- Erick de Oliveira SIlva - erickoliveira.eos@gmail.com

Inicialmente, devemos executar as células abaixo para realizarmos o upload dos arquivos necessários para a execução das linhas de código. Para isso, siga as instruções:

- Clone o repositório para o seu sistema local;
- Execute a célula abaixo;
- Clique em **"Browse"** e selecione o arquivo **dadosNE.zip**;
- Aguarde o arquivo carregar e execute as demais células abaixo até o tópico de Introdução



In [0]:
# Uploading files from your local file system

from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
!unzip dadosNE.zip

In [2]:
!pip install folium shapely

In [0]:
import os
import folium
import json
import pandas as pd
from branca.colormap import linear
import numpy as np
from shapely.geometry import Polygon
from shapely.geometry import Point
from numpy import random

# 1 - Introdução 

  
<p>Segundo a Organização Mundial de Saúde (<b>OMS</b>), saneamento é o controle de todos os fatores do meio físico do homem que exercem ou podem exercer efeitos nocivos sobre o bem estar físico, mental e social. É o conjunto de medidas adotadas em um local para melhorar a vida e a saúde dos habitantes, impedindo que fatores físicos de efeitos nocivos possam prejudicar as pessoas no seu bem-estar físico mental e social. No Brasil, esse conceito está previsto pela Lei Federal nº 11.445/07.<p>

<p> O Instituto Brasileiro de Geografia e Estatística disponibiliza em seu site diversos dados relativos ao Brasil. Nesse notebook nós acessamos dados referentes  ao percentual de esgotamento sanitário de cada um dos estados do Nordeste no ano de 2010 para gerar mapas temáticos usando o <b>Choropleth</b> e analisar se existem grandes diferenças entre os estado dessa mesma região. <p>
 

## 1.1 Importando os datasets

In [0]:
dataRN = pd.read_csv("data/esg_RN.csv")
dataPB = pd.read_csv("data/esg_PB.csv")
dataPE = pd.read_csv("data/esg_PE.csv")
dataCE = pd.read_csv("data/esg_CE.csv")
dataMA = pd.read_csv("data/esg_MA.csv")
dataPI = pd.read_csv("data/esg_PI.csv")
dataAL = pd.read_csv("data/esg_AL.csv")
dataSE = pd.read_csv("data/esg_SE.csv")
dataBA = pd.read_csv("data/esg_BA.csv")

Após importar os dados, podemos concatená-los em um único dataset:

In [0]:
data = pd.concat([dataRN, dataPB, dataPE, dataCE, dataMA, dataPI, dataAL, dataSE, dataBA], ignore_index=True)

Para alterar o nome das colunas:

In [0]:
data.columns = ["Cidade", "Esgotamento"]

In [7]:
data.head()

Cidade  Esgotamento
0           Acari         81.4
1             Açu         13.3
2  Afonso Bezerra         51.3
3       Água Nova          7.1
4      Alexandria         56.1

# 2 -  Importando os arquivos GeoJson

Existe um projeto  [Geojson](http://geojson.org/) contendo os perímetros de todos os municípios brasileiros agrupados por estado.(Fonte: https://github.com/tbrugz/geodata-br)

Para a região Nordeste, temos os seguintes dados:

* AL / Alagoas - [geojson/geojs-27-mun.json](geojson/geojs-27-mun.json)
* BA / Bahia - [geojson/geojs-29-mun.json](geojson/geojs-29-mun.json)
* CE / Ceará - [geojson/geojs-23-mun.json](geojson/geojs-23-mun.json)
* MA / Maranhão - [geojson/geojs-21-mun.json](geojson/geojs-21-mun.json)
* PB / Paraíba - [geojson/geojs-25-mun.json](geojson/geojs-25-mun.json)
* PE / Pernambuco - [geojson/geojs-26-mun.json](geojson/geojs-26-mun.json)
* PI / Piauí - [geojson/geojs-22-mun.json](geojson/geojs-22-mun.json)
* RN / Rio Grande do Norte - [geojson/geojs-24-mun.json](geojson/geojs-24-mun.json)
* SE / Sergipe - [geojson/geojs-28-mun.json](geojson/geojs-28-mun.json)

Com esses dados, é possível criar um mapa contendo as divisórias entre os estados, para que assim possamos utilizar, na próxima seção, separadamente os dados de cada estado para colorir o mapa com <b> Choropleth</b>, de acordo com o seu nível de saneamento.

In [0]:
# Buscando arquivos em geojson/geojs-xx-mun.json
MA = os.path.join('geojson', 'geojs-21-mun.json')
PI = os.path.join('geojson', 'geojs-22-mun.json')
CE = os.path.join('geojson', 'geojs-23-mun.json')
RN = os.path.join('geojson', 'geojs-24-mun.json')
PB = os.path.join('geojson', 'geojs-25-mun.json')
PE = os.path.join('geojson', 'geojs-26-mun.json')
AL = os.path.join('geojson', 'geojs-27-mun.json')
SE = os.path.join('geojson', 'geojs-28-mun.json')
BA = os.path.join('geojson', 'geojs-29-mun.json')

nordeste = os.path.join('geojson', 'geojs-NE-mun.json')

# load the data and use 'latin-1'encoding because the accent
geo_json_NE = json.load(open(nordeste,encoding='latin-1'))

geo_json_MA = json.load(open(MA,encoding='latin-1'))
geo_json_PI = json.load(open(PI,encoding='latin-1'))
geo_json_CE = json.load(open(CE,encoding='latin-1'))
geo_json_RN = json.load(open(RN,encoding='latin-1'))
geo_json_PB = json.load(open(PB,encoding='latin-1'))
geo_json_PE = json.load(open(PE,encoding='latin-1'))
geo_json_AL = json.load(open(AL,encoding='latin-1'))
geo_json_SE = json.load(open(SE,encoding='latin-1'))
geo_json_BA = json.load(open(BA,encoding='latin-1'))

In [0]:
# http://cidades.ibge.gov.br/painel/historico.php?codmun=241030
# Presidente Juscelino city changes your name to Serra Caiada
geo_json_RN['features'][112]['properties']['description'] = 'Serra Caiada'
geo_json_RN['features'][112]['properties']['name'] = 'Serra Caiada'

In [10]:
# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=5,
    tiles='Stamen Terrain'
)

# Configure geojson layer
folium.GeoJson(geo_json_NE).add_to(m)

m

# 3 - Criando mapas temáticos com o Choropleth

Agora, temos definidas as linhas divisórias no mapa e possuímos um <b> Dataframe </b> com os níveis de Esgotamente sanitário de cada cidade do Nordeste. Assim nesta seção, os utilizaremos para colorir o mapa.

In [11]:
data.head()

Cidade  Esgotamento
0           Acari         81.4
1             Açu         13.3
2  Afonso Bezerra         51.3
3       Água Nova          7.1
4      Alexandria         56.1

## 3.1 Criando uma escala de cor para o Choropleth

In [12]:
# colormap yellow and green (YlGn)
colormap = linear.YlGn_03.scale(
    data["Esgotamento"].min(),
    data["Esgotamento"].max())

print(colormap(5000.0))

colormap

#31a354


## 3.2 Conversão de Dataframe para Dicionário

Convertemos o Dataframe em Dicionário para que os nomes das cidades sejam utilizadas como chaves para seu próprio nível de Esgostamento Sanitário.

In [13]:
# Converter para dicionário
esg_dict = data.set_index('Cidade')['Esgotamento']
esg_dict[:100]

Cidade
Acari                      81.4
Açu                        13.3
Afonso Bezerra             51.3
Água Nova                   7.1
Alexandria                 56.1
Almino Afonso              12.8
Alto do Rodrigues           8.4
Angicos                    38.3
Antônio Martins            24.6
Apodi                      10.5
Areia Branca               29.4
Arês                       20.8
Augusto Severo             16.6
Baía Formosa               10.1
Baraúna                     9.8
Barcelona                  23.7
Bento Fernandes            14.4
Bodó                       40.9
Bom Jesus                  10.8
Brejinho                   32.3
Caiçara do Norte           40.5
Caiçara do Rio do Vento    56.8
Caicó                      76.3
Campo Redondo              43.5
Canguaretama                5.4
Caraúbas                   12.7
Carnaúba dos Dantas        16.3
Carnaubais                 45.3
Ceará-Mirim                54.5
Cerro Corá                  4.8
                           ... 
L

In [15]:
# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=6,
    tiles='Stamen Terrain'
)

# create a threshold of legend
threshold_scale = np.linspace(data['Esgotamento'].min(),
                              data['Esgotamento'].max(), 6, dtype=int).tolist()

m.choropleth(
    geo_data=geo_json_NE,
    data=data,
    columns=['Cidade', 'Esgotamento'],
    key_on='feature.properties.description',
    fill_color='BuGn',
    legend_name='Esgotamento sanitário adequado (2010)',
    highlight=True,
    threshold_scale = threshold_scale
)

m

## 3.3 Saneamento no RN



Nesta subseção restringiremos a análise para a realidade potiguar em 2010, ordenando as cidades da que possui maior nível (porcentagem) de saneamento para a que possui menor.

In [0]:
dataRN.columns = ['Cidade', 'Esgotamento']

In [17]:
test = dataRN.sort_values(['Esgotamento'],ascending=False)
test

Cidade  Esgotamento
157     Timbaúba dos Batistas         97.3
33              Currais Novos         82.2
0                       Acari         81.4
122                Santa Cruz         81.2
22                      Caicó         76.3
153                   Tangará         75.4
165                    Viçosa         73.4
146      Serra Negra do Norte         73.3
62           Jardim do Seridó         73.1
75                   Lucrécia         72.4
42                   Florânia         72.3
96                   Parelhas         70.1
78                      Macau         70.0
127       São Bento do Trairí         68.9
126        São Bento do Norte         67.4
32                    Cruzeta         65.4
130   São Gonçalo do Amarante         65.1
87                    Mossoró         64.6
134        São José do Seridó         63.9
124         Santana do Seridó         63.1
88                      Natal         61.8
107                Pendências         61.7
139                São Rafael         60.2
73                      Lajes         59.9
118                 Riachuelo         59.3
98               Passa e Fica         58.1
116            Riacho da Cruz         57.9
123          Santana do Matos         57.8
137      São Paulo do Potengi         57.7
102            Pau dos Ferros         57.5
..                        ...          ...
163                 Venha-Ver          3.6
164                 Vera Cruz          3.4
103              Pedra Grande          3.4
54                       Itaú          3.4
138                 São Pedro          2.9
136     São Miguel do Gostoso          2.8
91      Olho d'Água do Borges          2.4
72              Lagoa Salgada          2.1
52                    Ipueira          2.0
166                 Vila Flor          1.7
44             Frutuoso Gomes          1.7
143  Senador Georgino Avelino          1.6
145              Serra do Mel          1.4
113                    Pureza          1.3
161                   Upanema          1.3
159          Triunfo Potiguar          1.3
101               Santa Maria          1.2
151          Taboleiro Grande          1.2
110                Portalegre          1.1
43           Francisco Dantas          1.1
93                     Paraná          1.1
121               Ruy Barbosa          0.9
45                   Galinhos          0.9
97                Rio do Fogo          0.8
148       Serrinha dos Pintos          0.7
55                     Jaçanã          0.7
129    São Francisco do Oeste          0.5
81                    Martins          0.3
31        Coronel João Pessoa          0.3
111           Porto do Mangue          0.2

[167 rows x 2 columns]

In [18]:
# Create a map object
rn = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=7,
    tiles='Stamen Terrain'
)

# create a threshold of legend
threshold_scale = np.linspace(data['Esgotamento'].min(),
                              data['Esgotamento'].max(), 6, dtype=int).tolist()

rn.choropleth(
    geo_data=geo_json_RN,
    data=data,
    columns=['Cidade', 'Esgotamento'],
    key_on='feature.properties.description',
    fill_color='YlGn',
    legend_name='Esgotamento sanitário adequado no RN (2010)',
    highlight=True,
    threshold_scale = threshold_scale
)

rn

# 4 - Comparação com Reportagens atuais

Para este notebook foram usadas as reportagens disponíveis nos seguintes endereços:

<h3> Seminário: 120 anos de Legislação Sanitária (Dezembro de 2014) </h3>
  
  <a href="http://www.cvs.saude.sp.gov.br/up/MonicaPorto.pdf"> Os Desafios do Setor de Saneamento do Brasil  </a>
  
 Neste primeiro link é apresentado um estudo sobre a situação sanitária do Brasil com foco em São Paulo, nos quais são apresentados 3 entraves principais para a má situação sanitária:
 
 - Dificuldades de carácter técnico;
 - Dificuldades de carácter financeiro;
 - Dificuldades diversas: gestão e capacitação técnica.
 
 Sobre o primeiro entrave é destacado a dificuldade de implantação em áreas nas quais houve crescimento desordenado. Já para o segundo é destacado o  investimento de <b> R$ 8,32 bilhões </b> entre 2009 e 2013, e o que se obteve de resultados foi (fonte: De Olho no PAC, Instituto Trata Brasil):
 - 19% das obras concluídas;
 - 19% das obras em andamento normal;
 - 23% das obras paralisadas
 - 22% das obras atrasadas;
 - 3% das obras recém iniciadas;
 - 13% das obras não iniciadas
 
Sendo assim, é possível inferir que se em 4 anos foi possível apenas a conclusão de 19% das obras há problemas de gestão nas politícas de saneamento, além disso também é destacado a falta de capacidade técnicas dos municipios e empresas contratadas por eles, atrasando ainda mais o anadamento das obras.

<h3> G1 - Natureza  (01/05/2015) </h3>

<a href="http://g1.globo.com/natureza/blog/nova-etica-social/post/falta-de-saneamento-basico-no-brasil-uma-verdade-mais-do-que-inconveniente.html"> Falta de saneamento básico no Brasil: uma verdade mais do que inconveniente  </a>

Neste segundo link, é feita uma entrevista com o presidente do <a href="http://www.tratabrasil.org.br/"> Instituto Trata Brasil </a>, Edison Carlos que na época da entrevista, havia acabado de lançar um novo ranking de saneamento básico no Brasil. 

Ao longo da entrevista são descritos os desafios encontrados para a evolução do saneamento, e explicadas as ações promovidas pelo instituto para a conscientização. Apesar ser citado um estudo, não foi possível encontra-lo pelo link da reportagem.

<h3> UOL Notícias - Ciência de Saúde (19/08/2018) </h3>

<a href="https://noticias.uol.com.br/saude/ultimas-noticias/redacao/2018/09/19/saneamento-basico-doencas-dengue-chinkungunya-ibge.htm">No Brasil, 35% das cidades sofrem com doenças ligadas a falta de saneamento. </a> 

Por último temos uma reportagem de 2018 que retrata e liga diretamente epidemias ligadas ao saneamento básico,  os municípios relatam ter registrado epidemias ou endemias de diarreia, leptospirose, verminoses, cólera, difteria, zika, chikungunya, tifo, malária, hepatite, febre amarela, dermatite, dentre outras. Ainda segundo a mesma:

<h4> "Os dados do IBGE mostram uma grande disparidade entre regiões do país em termos de políticas para o saneamento. Enquanto na região Sul 63,7% dos municípios dizem ter o plano, só 18,6% dos municípios do Nordeste possuem esse tipo de instrumento." </h4>

E ainda é complementado com o fato sobre o número de municípios que possuem um plano de saneamento aumentou 35,4% em relação à última pesquisa feita pelo IBGE, em 2011.

# 5 - Conclusão

Diante de toda a situação, é possível afirmar que o Brasil não está com bons indices de saneamento. Ou seja, mesmo com o plano nacional de metas para 20 anos, os municipios não conseguem se adequar e por consequência a população sofre.com doenças e descaso.